In [5]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, f1_score, cohen_kappa_score, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_validate
from scipy.stats import loguniform
import sys
from IPython.display import display, HTML
import importlib
import warnings

In [4]:
warnings.filterwarnings('ignore')

In [6]:
sys.path.append('../scripts')

In [7]:
import scoring_and_evaluation
import feature_preprocessing

In [8]:
importlib.reload(scoring_and_evaluation)
importlib.reload(feature_preprocessing)

<module 'feature_preprocessing' from 'C:\\Users\\Jorge\\Desktop\\Data_Science\\Projects\\Depression_repo\\notebooks\\../scripts\\feature_preprocessing.py'>

In [9]:
from preprocessing import load_list_from_pkl
from scoring_and_evaluation import loguniform_int, adjacent_accuracy_score, randomized_search_and_cv
from feature_preprocessing import preprocess_dataframe

Column name lists and dataframe information loading:

In [10]:
list_path = '../data/lists/'
base_feats_cat = load_list_from_pkl(list_path+'base_feats_cat.pkl')
base_feats_num = load_list_from_pkl(list_path+'base_feats_num.pkl')
stats_cols = load_list_from_pkl(list_path+'stats_cols.pkl')
metadata_file = '../data/processed/metadata.csv'
df_path = '../data/processed/'
metadata_df = pd.read_csv(metadata_file)
display(HTML(metadata_df.to_html()))

,DataFrame,Shape,Comments
0,df_input,"(33320, 127)",Complete cleaned input data
1,df_study,"(9227, 87)",Cleaned + filtered input data based on paper's specifications + deleted features missing >10% data
2,data,"(9227, 83)",df_study without target variables
3,target_cat,"(9227,)",target phq9_cat_end from df_study
4,data_train,"(8304, 83)",90% of df data for model training
5,data_test,"(923, 83)",10% of df data for model testing
6,target_train,"(8304,)",90% of df target_cat for model training
7,target_test,"(923,)",10% of df target_cat for model testing
8,X_train_full,"(8304, 143)",data_train df with all additional feature engineering columns (+50)
9,X_train_full_preprocessed,"(8304, 145)","X_train_full df with preprocessing (imputation, scaling...)"


Dataframe loading:

In [11]:
data = pd.read_pickle(r'../data/processed/data')
data_add_features = pd.read_pickle(r'../data/processed/data_add_features')
target_cat = pd.read_pickle(r'../data/processed/target_cat')

Scorer creation, which allows to compare the results with the results obtained by the authors in the paper:

In [12]:
custom_f1 = make_scorer(f1_score, greater_is_better=True, average="weighted")
adjacent_accuracy = make_scorer(adjacent_accuracy_score, greater_is_better=True)
cohen_kappa = make_scorer(cohen_kappa_score, greater_is_better=True, weights='quadratic')

In [13]:
scoring = {'custom_f1': custom_f1,
           'adjacent_accuracy_score': adjacent_accuracy,
           'cohen_kappa_score': cohen_kappa
          }

# Modeling: Logistic Regression, HistGradientBoostingClassifier and XGBoost

As a baseline, a `LogisticRegression()` model is used, with 3 hyperparameters to be tuned while also performing a generalization scoring. These two steps are performed through a nested cross-validation.

## Logistic Regression

In [14]:
model_LR = LogisticRegression()

In [15]:
param_distributions = {
    "max_iter": loguniform_int(1000, 5000),
    "C": loguniform(0.001, 1),
    "solver": ['newton-cg', 'lbfgs', 'liblinear']
}

The nested cross-validation procedure allows to optimize hyperparameters (in this case with `RandomizedSearchCV`) while testing the prediction generalization capabilities of a model using a k-fold cross-validation (with `cross_validate`). The hyperparameter search is therefore performed on each of the folds of the outer splits of the `cross_validate` step. The idea of this procedure is to avoid an optimistic bias on the performance evaluation and/or avoid overfitting.
The `RandomizedSearchCV` allows for a stochastic search of the hyperparameter values and combinations by specifying a distribution of each hyperparameter value that is searched. An inner cross-validation is performed where the data is split according to the value of `cv`, and in each split there are `n_iter` model fittings with different hyperparameter combinations.
Then, for each fold in the outer `cross_validate`, the best model with the hyperparameter values selected is chosen to score its generalization performance. the outer cross-validation allows to score the model on data it has not been trained on. One of the drawbacks of this methodology is that the number of model fitting can quickly increase significantly, which translates into more comkputing resources and time. <br>
Given that we are using multiple scorers, the `refit` parameter is used with the f1 scorer. This allows to use that specific scorer to find the best parameters. This is a different methodology than used in the paper, where the authors used the Cohen-Kappa score as the one to be optimized.

For this baseline study, we will also compare the performance of the models on 3 datasets: the original dataset with some basic preprocessing (preprocess_dataframe(data)), the same dataset with all additional features from the feature engineering process (preprocess_dataframe(data_add_features)) and finally the original dataset with only some pre-selected basic categorical and numerical features (preprocess_dataframe(data[base_feats_cat+base_feats_num])).

To observe the results, the function [randomized_search_and_cv()](https://github.com/jloayza10/depression_prediction_project/blob/main/scripts/scoring_and_evaluation.py) is used. Both the inner and outher cross-validations use a 5-fold split. The function prints the generalization scores by averaging the 5 results obtained on each of the outer folds. Also, for each of these 5 folds, the inner cross-validation performs 5 splits (80%-20% of the remaining data) and performs 20 iterations to find the best hyperparameter values. These results are also printed.   <br>
If we look more in detail the hyperparameter tuning step for each of the 5 folds of the outer cross-validation, the results are presented below. This allows to see if the chosen hyperparameter values are always similar or if the search has varied a lot:

In [498]:
cv_results_data = randomized_search_and_cv(model_LR, param_distributions, preprocess_dataframe(data), target_cat, scoring)

'LogisticRegression()' generalization scores:
Weighted f1: 0.418 +/- 0.010
Cohen Kappa: 0.424 +/- 0.017
Adjacent Accuracy: 0.825 +/- 0.008

Best hyperparameters for fold 1 with a best result of 0.4161621108196994
{'C': 0.7475987911298642, 'max_iter': 2358, 'solver': 'newton-cg'}
Best hyperparameters for fold 2 with a best result of 0.4099172204380227
{'C': 0.11938997108268634, 'max_iter': 4097, 'solver': 'newton-cg'}
Best hyperparameters for fold 3 with a best result of 0.4141469441333224
{'C': 0.11938997108268634, 'max_iter': 4097, 'solver': 'newton-cg'}
Best hyperparameters for fold 4 with a best result of 0.40714033900214497
{'C': 0.7475987911298642, 'max_iter': 2358, 'solver': 'newton-cg'}
Best hyperparameters for fold 5 with a best result of 0.4099028800053205
{'C': 0.11938997108268634, 'max_iter': 4097, 'solver': 'newton-cg'}


Results show that 3 out of the 5 models converged to the same hyperparameters, while the other 2 converged to the same solver but different C and max_iter values. However, all f1-scores are almost equivalent.

In [501]:
cv_results_base = randomized_search_and_cv(model_LR, param_distributions, preprocess_dataframe(data[base_feats_cat+base_feats_num]), target_cat, scoring)

'LogisticRegression()' generalization scores:
Weighted f1: 0.420 +/- 0.011
Cohen Kappa: 0.414 +/- 0.019
Adjacent Accuracy: 0.825 +/- 0.006

Best hyperparameters for fold 1 with a best result of 0.4149053137484504
{'C': 0.7475987911298642, 'max_iter': 2358, 'solver': 'newton-cg'}
Best hyperparameters for fold 2 with a best result of 0.41511789167224966
{'C': 0.5519326346081438, 'max_iter': 2087, 'solver': 'lbfgs'}
Best hyperparameters for fold 3 with a best result of 0.41353635153989377
{'C': 0.7475987911298642, 'max_iter': 2358, 'solver': 'newton-cg'}
Best hyperparameters for fold 4 with a best result of 0.41478763223582205
{'C': 0.7475987911298642, 'max_iter': 2358, 'solver': 'newton-cg'}
Best hyperparameters for fold 5 with a best result of 0.41421443827724724
{'C': 0.11938997108268634, 'max_iter': 4097, 'solver': 'newton-cg'}


In [506]:
cv_results_LR_full = randomized_search_and_cv(model_LR, param_distributions, preprocess_dataframe(data_add_features), target_cat, scoring)

'LogisticRegression()' generalization scores:
Weighted f1: 0.426 +/- 0.015
Cohen Kappa: 0.437 +/- 0.019
Adjacent Accuracy: 0.832 +/- 0.009

Best hyperparameters for fold 1 with a best result of 0.41609777106229123
{'C': 0.11938997108268634, 'max_iter': 4097, 'solver': 'newton-cg'}
Best hyperparameters for fold 2 with a best result of 0.41454042634241617
{'C': 0.7475987911298642, 'max_iter': 2358, 'solver': 'newton-cg'}
Best hyperparameters for fold 3 with a best result of 0.4157694840228783
{'C': 0.015478974396671013, 'max_iter': 1866, 'solver': 'newton-cg'}
Best hyperparameters for fold 4 with a best result of 0.4144613865885335
{'C': 0.7475987911298642, 'max_iter': 2358, 'solver': 'newton-cg'}
Best hyperparameters for fold 5 with a best result of 0.4199091692721623
{'C': 0.11938997108268634, 'max_iter': 4097, 'solver': 'newton-cg'}


## HistGradientBoostingClassifier

In [16]:
model_HGBC = HistGradientBoostingClassifier()

In [17]:
param_distributions_HGBC = {
    "learning_rate": loguniform(0.001, 2),
    "max_iter": loguniform_int(10, 1000),
    "max_leaf_nodes": loguniform_int(2, 100),
    "max_depth": loguniform_int(2, 50),
    "max_bins": loguniform_int(2, 100),
    "l2_regularization": loguniform(0.0001, 0.5)
}

In [497]:
cv_results_HGBC_data = randomized_search_and_cv(model_HGBC, param_distributions_HGBC, preprocess_dataframe(data), target_cat, scoring)

'HistGradientBoostingClassifier()' generalization scores:
Weighted f1: 0.428 +/- 0.008
Cohen Kappa: 0.411 +/- 0.011
Adjacent Accuracy: 0.825 +/- 0.006

Best hyperparameters for fold 1 with a best result of 0.40994411753210735
{'l2_regularization': 0.02853359428016051, 'learning_rate': 0.050080009587496074, 'max_bins': 80, 'max_depth': 13, 'max_iter': 640, 'max_leaf_nodes': 3}
Best hyperparameters for fold 2 with a best result of 0.41449792675075126
{'l2_regularization': 0.02853359428016051, 'learning_rate': 0.050080009587496074, 'max_bins': 80, 'max_depth': 13, 'max_iter': 640, 'max_leaf_nodes': 3}
Best hyperparameters for fold 3 with a best result of 0.41350562774139005
{'l2_regularization': 0.00023109044472742628, 'learning_rate': 0.024552114543090666, 'max_bins': 84, 'max_depth': 11, 'max_iter': 241, 'max_leaf_nodes': 6}
Best hyperparameters for fold 4 with a best result of 0.41195800553360185
{'l2_regularization': 0.00023109044472742628, 'learning_rate': 0.024552114543090666, 'max_

In [495]:
cv_results_HGBC_base = randomized_search_and_cv(model_HGBC, param_distributions_HGBC, preprocess_dataframe(data[base_feats_cat+base_feats_num]), target_cat, scoring)

'HistGradientBoostingClassifier()' generalization scores:
Weighted f1: 0.435 +/- 0.003
Cohen Kappa: 0.442 +/- 0.021
Adjacent Accuracy: 0.836 +/- 0.008

Best hyperparameters for fold 1 with a best result of 0.4093971740378257
{'l2_regularization': 0.02853359428016051, 'learning_rate': 0.050080009587496074, 'max_bins': 80, 'max_depth': 13, 'max_iter': 640, 'max_leaf_nodes': 3}
Best hyperparameters for fold 2 with a best result of 0.41025388113630223
{'l2_regularization': 0.00023109044472742628, 'learning_rate': 0.024552114543090666, 'max_bins': 84, 'max_depth': 11, 'max_iter': 241, 'max_leaf_nodes': 6}
Best hyperparameters for fold 3 with a best result of 0.41312499098101485
{'l2_regularization': 0.02853359428016051, 'learning_rate': 0.050080009587496074, 'max_bins': 80, 'max_depth': 13, 'max_iter': 640, 'max_leaf_nodes': 3}
Best hyperparameters for fold 4 with a best result of 0.41210720019525093
{'l2_regularization': 0.02853359428016051, 'learning_rate': 0.050080009587496074, 'max_bins

In [496]:
cv_results_HGBC_full = randomized_search_and_cv(model_HGBC, param_distributions_HGBC, preprocess_dataframe(data_add_features), target_cat, scoring)

'HistGradientBoostingClassifier()' generalization scores:
Weighted f1: 0.431 +/- 0.016
Cohen Kappa: 0.432 +/- 0.027
Adjacent Accuracy: 0.830 +/- 0.008

Best hyperparameters for fold 1 with a best result of 0.4157359701513313
{'l2_regularization': 0.02853359428016051, 'learning_rate': 0.050080009587496074, 'max_bins': 80, 'max_depth': 13, 'max_iter': 640, 'max_leaf_nodes': 3}
Best hyperparameters for fold 2 with a best result of 0.42000243902431655
{'l2_regularization': 0.21957037777542873, 'learning_rate': 0.07829484945755741, 'max_bins': 2, 'max_depth': 14, 'max_iter': 45, 'max_leaf_nodes': 15}
Best hyperparameters for fold 3 with a best result of 0.4221289313308582
{'l2_regularization': 0.02853359428016051, 'learning_rate': 0.050080009587496074, 'max_bins': 80, 'max_depth': 13, 'max_iter': 640, 'max_leaf_nodes': 3}
Best hyperparameters for fold 4 with a best result of 0.41673924662812933
{'l2_regularization': 0.00023109044472742628, 'learning_rate': 0.024552114543090666, 'max_bins': 

## XGBoost

In [18]:
model_xgb = XGBClassifier(use_label_encoder=False,error_score='raise', n_jobs=3)

In [19]:
param_distributions_xgb = {
    "n_estimators": loguniform_int(10, 5000),
    "max_depth": loguniform_int(1, 20),
    "learning_rate": loguniform(0.0001, 1)
}

In [20]:
cv_results_xgb_data = randomized_search_and_cv(model_xgb, param_distributions_xgb, preprocess_dataframe(data), target_cat, scoring)

'XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, error_score='raise', gamma=None,
              gpu_id=None, importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=3, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=False,
              validate_parameters=None, verbosity=None)' generalization scores:
Weighted f1: 0.430 +/- 0.005
Cohen Kappa: 0.406 +/- 0.023
Adjacent Accuracy: 0.825 +/- 0.006

Best hyperparameters for fold 1 with a best result of 0.40873389830669626
{'learning_rate': 0.04806954824961597, 'max_depth': 3, 'n_estima

In [21]:
cv_results_xgb_base = randomized_search_and_cv(model_xgb, param_distributions_xgb, preprocess_dataframe(data[base_feats_cat+base_feats_num]), target_cat, scoring)

'XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, error_score='raise', gamma=None,
              gpu_id=None, importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=3, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=False,
              validate_parameters=None, verbosity=None)' generalization scores:
Weighted f1: 0.437 +/- 0.013
Cohen Kappa: 0.432 +/- 0.031
Adjacent Accuracy: 0.836 +/- 0.007

Best hyperparameters for fold 1 with a best result of 0.40134390726365066
{'learning_rate': 0.014297724879798386, 'max_depth': 3, 'n_estim

In [22]:
cv_results_xgb_full = randomized_search_and_cv(model_xgb, param_distributions_xgb, preprocess_dataframe(data_add_features), target_cat, scoring)

'XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, error_score='raise', gamma=None,
              gpu_id=None, importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=3, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, use_label_encoder=False,
              validate_parameters=None, verbosity=None)' generalization scores:
Weighted f1: 0.428 +/- 0.015
Cohen Kappa: 0.421 +/- 0.023
Adjacent Accuracy: 0.830 +/- 0.004

Best hyperparameters for fold 1 with a best result of 0.4101007027061435
{'learning_rate': 0.014297724879798386, 'max_depth': 3, 'n_estima